In [2]:
##########################################################
# <실습20> 챗봇 엔진 동작
##########################################################
import sys
sys.path.append('C:/chatbot')

from config.DatabaseConfig import *
from utils.Database import Database
from utils.Preprocess import Preprocess

# 전처리 객체 생성
p = Preprocess(word2index_dic='../train_tools/dict/chatbot_dict.bin',
               userdic='../utils/user_dic.tsv')

In [3]:
# 질문/답변 학습 DB 연결 객체 생성
db = Database(
    host=DB_HOST, 
    user=DB_USER,
    password=DB_PASSWORD,
    db_name=DB_NAME
)
db.connect()

chatbot 연결 시작
DB 연결 성공!!!


In [19]:
# 원문
# query = "오전에 탕수육 10개 주문합니다"
# query = "화자의 질문 의도를 파악합니다."
# query = "안녕하세요"
query = "8시까지 자장면 2그릇 주문할게요"

# Intent(의도) 파악
from models.intent.IntentModel import IntentModel
intent = IntentModel(model_name='../models/intent/intent_model.h5', preprocess=p)

In [20]:
predict = intent.predict_class(query)
predict

2

In [21]:
intent_name = intent.labels[predict]
intent_name

'주문'

In [22]:
### 개체명 인식
from models.ner.NerModel import NerModel
ner = NerModel(model_name='../models/ner/ner_model.h5', preprocess=p)
predicts = ner.predict(query)
predicts

[('8시', 'B_DT'), ('자장면', 'B_FOOD'), ('2', 'O'), ('그릇', 'O'), ('주문', 'O')]

In [23]:
ner_tags = ner.predict_tags(query)
ner_tags

['B_DT', 'B_FOOD']

In [24]:
print("질문 : ", query)
print("=" * 80)
print("의도 파악 : ", intent_name)
print("개체명 인식 : ", predicts)
print("답변 검색에 필요한 NER 태그 : ", ner_tags)
print("=" * 80)

질문 :  8시까지 자장면 2그릇 주문할게요
의도 파악 :  주문
개체명 인식 :  [('8시', 'B_DT'), ('자장면', 'B_FOOD'), ('2', 'O'), ('그릇', 'O'), ('주문', 'O')]
답변 검색에 필요한 NER 태그 :  ['B_DT', 'B_FOOD']


In [25]:
## DB에서 답변 검색
from utils.FindAnswer import FindAnswer

try:
    f = FindAnswer(db)
    answer_text, answer_image = f.search(intent_name, ner_tags)
    answer = f.tag_to_word(predicts, answer_text)
except:
    answer = '죄송합니다. 무슨 말인지 모르겠어요~'

print("답변 : ", answer)

db.close()

============== Database.py ===================
## Database.py의 select_one함수, sql=
============== FindAnswer.py ===================
** intent_name = 주문
** ner_tags =  ['B_DT', 'B_FOOD']
** sql =  SELECT * FROM chatbot_train_data WHERE intent="주문" AND ( NER LIKE '%B_DT%' or  NER LIKE '%B_FOOD%' ) order by rand() limit 1
** answer = {'id': 4, 'intent': '주문', 'ner': 'B_FOOD', 'query': '{B_FOOD} 주문할게요', 'answer': '{B_FOOD} 주문 처리 감사!!', 'answer_image': None}
answer= {'id': 4, 'intent': '주문', 'ner': 'B_FOOD', 'query': '{B_FOOD} 주문할게요', 'answer': '{B_FOOD} 주문 처리 감사!!', 'answer_image': None}
답변 :  자장면 주문 처리 감사!!
DB가 닫혔습니다.
